In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

import gc
from keras.models import Model
from keras.layers import Input, Dense

from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Input, BatchNormalization, Dense, Reshape, Lambda, Dropout
from keras import metrics
#from . import backend as K
from keras import backend as K

Using TensorFlow backend.


# Keras Model

In [3]:

train = pd.read_csv('train_2016_v2.csv', nrows=13200) 
prop = pd.read_csv('properties_2016.csv', nrows=13200) 
sample = pd.read_csv('sample_submission.csv')


In [4]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [5]:
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015,239695.0,5725.17,NaN,NaN,NaN


In [40]:
prop['propertycountylandusecode'] = prop['propertycountylandusecode'].apply(lambda x: str(x))
encoder = preprocessing.LabelEncoder()
encoder.fit(prop['propertycountylandusecode'])
prop['propertycountylandusecode'] = encoder.transform(prop['propertycountylandusecode'])

prop['propertyzoningdesc'] = prop['propertyzoningdesc'].apply(lambda x: str(x))
encoder2 = preprocessing.LabelEncoder()
encoder2.fit(prop['propertyzoningdesc'])
prop['propertyzoningdesc'] = encoder2.transform(prop['propertyzoningdesc'])


In [42]:
# Discard all non-numeric data
prop = prop.select_dtypes([np.number])
train = train.select_dtypes([np.number])
sample = sample.select_dtypes([np.number])

gc.collect()


x_train = prop.drop(['parcelid'], axis=1)

gc.collect()


0

In [43]:
# Store for prediction phase
train_columns = x_train.columns

# Rows are not ordered, so join on parcelid
temp = pd.merge(left=train, right=prop, on=('parcelid'), how='outer')

# Casually fill in missing data with junk
temp = temp.fillna(0)

# Convert to numpy array for Keras
x_train = temp.drop(['parcelid', 'logerror'], axis=1).values
y_train = temp['logerror'].values

gc.collect()

14

In [44]:
scaler = preprocessing.StandardScaler()
# x_train.apply(lambda x: (x - x.mean()) / (x.max() - x.min()))
x_train = scaler.fit_transform(x_train)

# Normalize (across the whole dataframe cos we dun care)
mean_x = x_train.mean().astype(np.float32)
std_x = x_train.std().astype(np.float32)

mean_y = y_train.mean().astype(np.float32)
std_y = y_train.std().astype(np.float32)


In [45]:
def normalize(x):
    return (x-mean_x)/std_x

def normalize_y(y):
    return (y-mean_y)/std_y

def de_normalize_y(y):


    return (y*std_y) + mean_y

y_train = normalize(y_train)


***Dense***  implements the operation: output = activation(dot(input, kernel) + bias) where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only applicable if use_bias is True).

***Dropout*** consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.

**lambda** Wraps arbitrary expression as a Layer object.

**Batch normalization layer** (Ioffe and Szegedy, 2014).
Normalize the activations of the previous layer at each batch, i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.(Reference https://arxiv.org/abs/1502.03167)



**ReLU** The Rectified Linear Unit has become very popular in the last few years. It computes the function f(x)=max(0,x)f(x)=max(0,x). In other words, the activation is simply thresholded at zero (see image above on the left). There are several pros and cons to using the ReLUs:(Reference http://cs231n.github.io/neural-networks-1/ by Stanford online).

https://github.com/Kulbear/deep-learning-nano-foundation/wiki/ReLU-and-Softmax-Activation-Functions


In [46]:
# Build a simple model
model = Sequential([
    #Lambda(normalize,input_shape=(52, )),
	Dense(60,input_shape=(54, )),
    BatchNormalization(),
    Dropout(0.08),
	Dense(160, activation='relu'),
	BatchNormalization(),
    Dropout(0.38),
    Dense(20, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])



In [48]:
model.compile(loss='mean_absolute_error', optimizer='adam')
model.fit(x_train, y_train, batch_size=24, epochs=15)

Epoch 1/15
26352/26352 [==============================] - 7s - loss: 0.1296     
Epoch 2/15
26352/26352 [==============================] - 5s - loss: 0.0378     
Epoch 3/15
26352/26352 [==============================] - 5s - loss: 0.0375     
Epoch 4/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 5/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 6/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 7/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 8/15
26352/26352 [==============================] - 5s - loss: 0.0373     
Epoch 9/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 10/15
26352/26352 [==============================] - 6s - loss: 0.0374     
Epoch 11/15
26352/26352 [==============================] - 6s - loss: 0.0374     
Epoch 12/15
26352/26352 [==============================] - 5s - loss: 0.0374     
Epoch 13/15
26352/26352 [

In [50]:
model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(x_train, y_train, batch_size=32, epochs=25)

Epoch 1/25
26352/26352 [==============================] - 6s - loss: 0.0172     
Epoch 2/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 3/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 4/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 5/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 6/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 7/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 8/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 9/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 10/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 11/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 12/25
26352/26352 [==============================] - 4s - loss: 0.0172     
Epoch 13/25
26352/26352 [

**fit**     Trains the model for a fixed number of epochs (iterations on a dataset).
**compile**  Configures the model for training.

In [51]:
# Prepare the submission data
sample['parcelid'] = sample['ParcelId']
del sample['ParcelId']

df_test = pd.merge(sample, prop, on='parcelid', how='left')
df_test = df_test.fillna(0)

x_test = df_test[train_columns]
#predict(self, x, batch_size=None, verbose=0, steps=None)
p_test = model.predict(x_test.values)
p_test = de_normalize_y(p_test)

In [52]:
p_test

array([[ 0.00804569],
       [ 0.00804569],
       [ 0.00804569],
       ..., 
       [ 0.00830373],
       [ 0.00830373],
       [ 0.00830374]], dtype=float32)

Returns the loss value & metrics values for the model in test mode.

Computation is done in batches.

In [53]:
# evaluate(self, x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None)
q_test=model.evaluate(x_train,y_train)
#q_test= de_normalize(q_test)


25824/26352 [============================>.] - ETA: 0s

In [54]:
q_test

0.017219735161938057

In [55]:
q_test= de_normalize_y(q_test)

In [56]:
q_test

0.010309619403134178

Runs a single gradient update on a single batch of data.



In [57]:
#train_on_batch(self, x, y, sample_weight=None, class_weight=None)
model.train_on_batch(x_train,y_train)

0.017224442

Test the model on a single batch of samples.



In [58]:
#test_on_batch(self, x, y, sample_weight=None)
model.test_on_batch(x_train,y_train, sample_weight=None)


0.017219124

Returns predictions for a single batch of samples.



In [59]:
#predict_on_batch(self, x)
model.predict_on_batch(x_train)

array([[ 0.01621497],
       [ 0.01621497],
       [ 0.01621497],
       ..., 
       [ 0.00096313],
       [ 0.00096735],
       [ 0.00104029]], dtype=float32)

In [18]:
#model.evaluate_on_batch(x_train,y_train)

**generator**
Fits the model on data yielded batch-by-batch by a Python generator.
The generator is run in parallel to the model, for efficiency. For instance, this allows you to do real-time data augmentation on images on CPU in parallel to training your model on GPU.

In [19]:
###fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

#model.fit_generator(x_train, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

In [20]:
#evaluate_generator(self, generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False)
#model.evaluate_generator(self, generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False)

In [21]:
#predict_generator(self, generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
#model.predict_generator(x_train, generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

Retrieves a layer based on either its name (unique) or index.

Indices are based on order of horizontal graph traversal (bottom-up).

In [22]:
#get_layer(self, name=None, index=None)
lay=model.get_layer(x_train.all())

In [23]:
print(lay)

None


**generate output result to output.csv**

In [60]:
sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('output.csv', index=False, float_format='%.5f')

In [61]:
sub.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.008046,0.008046,0.008046,0.008046,0.008046,0.008046
1,10759547,0.008046,0.008046,0.008046,0.008046,0.008046,0.008046
2,10843547,0.008046,0.008046,0.008046,0.008046,0.008046,0.008046
3,10859147,0.008046,0.008046,0.008046,0.008046,0.008046,0.008046
4,10879947,0.008046,0.008046,0.008046,0.008046,0.008046,0.008046


# Support vector Machine(optional)

This is not complete and requires modification

In [ ]:

from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import utils
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot



fig, subaxes = plt.subplots(1, 1, figsize=(7, 5))
this_C = 1.0
clf = SVC(kernel = 'linear', C=this_C).fit(x_train, y_train)
title = 'Linear SVC, C = {:.3f}'.format(this_C)
plot_class_regions_for_classifier_subplot(clf, X_train, y_train, None, None, title, subaxes)

In [ ]:
from sklearn.svm import LinearSVC


clf = LinearSVC().fit(x_train, y_train)

print('Accuracy of Linear SVC classifier on training set: {:.2f}'
     .format(clf.score(x_train, y_train)))
print('Accuracy of Linear SVC classifier on test set: {:.2f}'
     .format(clf.score(x_test, y_test)))

**Accuracy of Support Vector Machine classifier**

In [ ]:

from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1).fit(X_train, y_train)
svm.score(X_test, y_test)